In [1]:
from prophet import Prophet
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import joblib

In [2]:
df = pd.read_csv('http_metrics.csv' )

In [3]:
df.head()

,ds,name,instances,display_host,record_count,mult_records,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,web_response_time,throughput
0,2024-04-15 23:32:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,30.099163,0.000091,0.785708,2.049792,0.007541,3898.0
1,2024-04-15 23:33:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,32.763374,0.000103,3.822411,8.938635,0.008417,3917.5
2,2024-04-15 23:34:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,34.776276,0.000110,3.822411,16.068309,0.009546,3993.0
3,2024-04-15 23:35:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,33.072502,0.000110,3.659296,8.886156,0.009014,3991.5
4,2024-04-15 23:36:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,42.575750,0.000107,15.111876,135.348995,0.012279,3915.5


In [4]:
ts=df[['ds','throughput']]

In [5]:
ts['ds'] = pd.to_datetime(ts['ds'])

<ipython-input-5-adb4a517f4fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts['ds'] = pd.to_datetime(ts['ds'])


In [6]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43272 entries, 0 to 43271
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ds          43272 non-null  datetime64[ns]
 1   throughput  43272 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 676.2 KB


In [7]:
Prophet

prophet.forecaster.Prophet

In [8]:
ts.rename(columns={"throughput": "y"}, inplace=True)


<ipython-input-8-8530735359ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts.rename(columns={"throughput": "y"}, inplace=True)


In [9]:
model = Prophet(interval_width=0.99, weekly_seasonality=True, yearly_seasonality=False, daily_seasonality=True)
model.fit(ts)
forecast = model.predict(ts)

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5s0t0xhx/7ap0_6a7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5s0t0xhx/ggbsz4ky.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=22987', 'data', 'file=/tmp/tmp5s0t0xhx/7ap0_6a7.json', 'init=/tmp/tmp5s0t0xhx/ggbsz4ky.json', 'output', 'file=/tmp/tmp5s0t0xhx/prophet_modelr82t0c3d/prophet_model-20240525232443.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:24:43 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:25:12 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [10]:
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-04-15 23:32:00,3553.844904,2274.680984,5052.324350,3553.844904,3553.844904,105.808399,105.808399,105.808399,138.302728,138.302728,138.302728,-32.494329,-32.494329,-32.494329,0.0,0.0,0.0,3659.653303
1,2024-04-15 23:33:00,3553.961052,2218.908344,5060.636575,3553.961052,3553.961052,91.927713,91.927713,91.927713,124.553396,124.553396,124.553396,-32.625683,-32.625683,-32.625683,0.0,0.0,0.0,3645.888765
2,2024-04-15 23:34:00,3554.077201,2155.357234,5075.934470,3554.077201,3554.077201,78.007551,78.007551,78.007551,110.764485,110.764485,110.764485,-32.756934,-32.756934,-32.756934,0.0,0.0,0.0,3632.084751
3,2024-04-15 23:35:00,3554.193349,2117.813257,4968.106166,3554.193349,3554.193349,64.049884,64.049884,64.049884,96.937966,96.937966,96.937966,-32.888082,-32.888082,-32.888082,0.0,0.0,0.0,3618.243233
4,2024-04-15 23:36:00,3554.309497,2096.778748,5053.252828,3554.309497,3554.309497,50.056687,50.056687,50.056687,83.075813,83.075813,83.075813,-33.019126,-33.019126,-33.019126,0.0,0.0,0.0,3604.366184


In [11]:
# Предикт делаем на полную исходную выборку:
performance = pd.merge(ts, forecast[['ds','yhat_lower', 'yhat',  'yhat_upper']], on='ds')
performance.head()


,ds,y,yhat_lower,yhat,yhat_upper
0,2024-04-15 23:32:00,3898.0,2274.680984,3659.653303,5052.324350
1,2024-04-15 23:33:00,3917.5,2218.908344,3645.888765,5060.636575
2,2024-04-15 23:34:00,3993.0,2155.357234,3632.084751,5075.934470
3,2024-04-15 23:35:00,3991.5,2117.813257,3618.243233,4968.106166
4,2024-04-15 23:36:00,3915.5,2096.778748,3604.366184,5053.252828


In [12]:
performance.rename(columns={'y': 'y_real', 'yhat_lower': 'y_pred_lower', 'yhat': 'y_pred', 'yhat_upper': 'y_pred_upper'}, inplace=True)
performance = performance[['ds', 'y_pred_lower', 'y_pred', 'y_pred_upper', 'y_real']]
performance.head()

,ds,y_pred_lower,y_pred,y_pred_upper,y_real
0,2024-04-15 23:32:00,2274.680984,3659.653303,5052.324350,3898.0
1,2024-04-15 23:33:00,2218.908344,3645.888765,5060.636575,3917.5
2,2024-04-15 23:34:00,2155.357234,3632.084751,5075.934470,3993.0
3,2024-04-15 23:35:00,2117.813257,3618.243233,4968.106166,3991.5
4,2024-04-15 23:36:00,2096.778748,3604.366184,5053.252828,3915.5


In [13]:
# Вычисляем стандартное отклонение предсказанных значений
std_dev = performance['y_pred'].std()

# Определяем аномалии с более строгим порогом
performance['anomaly'] = performance.apply(
    lambda row: 1 if (row['y_real'] < (row['y_pred_lower'] - 2.5 * std_dev)) or (row['y_real'] > (row['y_pred_upper'] + 2.5 * std_dev)) else 0, axis=1)

anomalies = performance[performance['anomaly'] == 1].sort_values(by='ds')
anomalies.head()

,ds,y_pred_lower,y_pred,y_pred_upper,y_real,anomaly
2791,2024-04-17 22:03:00,3552.759438,5041.221713,6486.153502,9225.00000,1
2792,2024-04-17 22:04:00,3556.506011,5037.021490,6328.504957,9070.00000,1
4979,2024-04-19 10:43:00,3717.512665,5139.635108,6525.075734,504.06668,1
4980,2024-04-19 10:44:00,3755.696786,5139.483275,6581.302352,504.06668,1
4981,2024-04-19 10:45:00,3941.607379,5139.221007,6672.518930,504.06668,1


In [14]:
performance['color'] = np.where(performance['anomaly']== 1, 'red', 'green')
performance['name'] = np.where(performance['anomaly'] == 1, 'Anomaly','Within Confidence Interval')

In [15]:
# Преобразуем исходные данные в DataFrame для Plotly
performance['timestamp'] = performance['ds']
performance['target'] = performance['y_real']
ts_df = performance[['timestamp', 'target']]

# Создаем график временного ряда
fig = px.line(ts_df, x='timestamp', y='target', title='Time Series with Anomalies')

# Добавляем аномалии на график
anomaly_points = performance[performance['anomaly'] == 1]
fig.add_scatter(x=anomaly_points['timestamp'], y=anomaly_points['target'],
                mode='markers', marker=dict(color='red', size=10), name='Anomalies')


In [19]:
# Подготовка итогового DataFrame
result = performance[['ds', 'y_real', 'anomaly']]
result.rename(columns={'ds': 'timestamp', 'y_real': 'Throughput'}, inplace=True)
result['model'] = 'prophet'

result.rename(columns={'ds': 'timestamp', 'Throughput': 'value'}, inplace=True)
result['Модель'] = 'Prophet'
result['Метрика'] = 'Throughput'
result['is_anomaly']=result['anomaly']
result['is_anomaly']=result['is_anomaly'].astype(bool)
# Сохраняем результат в CSV файл
result = result[['Модель', 'Метрика', 'timestamp', 'value',  'is_anomaly']]
result.to_csv('Prophet_Throughput.csv', index=False)

print("Результаты модели сохранены в файл 'result.csv'")

<ipython-input-19-a538da97a9be>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-19-a538da97a9be>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-19-a538da97a9be>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-19-a538da97a9be>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

Результаты модели сохранены в файл 'result.csv'


In [20]:
result

,Модель,Метрика,timestamp,value,is_anomaly
0,Prophet,Throughput,2024-04-15 23:32:00,3898.0,False
1,Prophet,Throughput,2024-04-15 23:33:00,3917.5,False
2,Prophet,Throughput,2024-04-15 23:34:00,3993.0,False
3,Prophet,Throughput,2024-04-15 23:35:00,3991.5,False
4,Prophet,Throughput,2024-04-15 23:36:00,3915.5,False
...,...,...,...,...,...
43267,Prophet,Throughput,2024-05-16 00:54:00,2560.0,False
43268,Prophet,Throughput,2024-05-16 00:55:00,2491.0,False
43269,Prophet,Throughput,2024-05-16 00:56:00,2429.0,False
43270,Prophet,Throughput,2024-05-16 00:57:00,2373.5,False


In [18]:
result['anomaly'].nunique()

KeyError: 'anomaly'

In [ ]:
joblib.dump(model, 'prophet_web_responce.pkl')